In [ ]:
from jupyter_cadquery import show, open_viewer, set_defaults
from build123d import *

cv = open_viewer("Build123d", cad_width=780, glass=True)

set_defaults(edge_accuracy=0.0001)

In [ ]:
with BuildPart() as b:
    Box(1, 1, 1)
    CounterBoreHole(0.2, 0.3, 0.1)
    Box(1, 0.2, 1, mode=Mode.SUBTRACT)
    Box(0.2, 1, 1, mode=Mode.SUBTRACT)

with BuildSketch() as s:
    with BuildLine() as line:
        l = Line((0, 0), (1, 0))
        Line(l @ 1, (1, 1))
        offset(line.line.edges(), amount=1)
    make_face()

with BuildLine() as l:
    l1 = Polyline((0.0000, 0.0771), (0.0187, 0.0771), (0.0094, 0.2569))
    l2 = Polyline((0.0325, 0.2773), (0.2115, 0.2458), (0.1873, 0.3125))
    RadiusArc(l1 @ 1, l2 @ 0, 0.0271)
    l3 = Polyline((0.1915, 0.3277), (0.3875, 0.4865), (0.3433, 0.5071))
    TangentArc(l2 @ 1, l3 @ 0, tangent=l2 % 1)
    l4 = Polyline((0.3362, 0.5235), (0.375, 0.6427), (0.2621, 0.6188))
    SagittaArc(l3 @ 1, l4 @ 0, 0.003)
    l5 = Polyline((0.2469, 0.6267), (0.225, 0.6781), (0.1369, 0.5835))
    ThreePointArc(l4 @ 1, (l4 @ 1 + l5 @ 0) * 0.5 + Vector(-0.002, -0.002), l5 @ 0)
    l6 = Polyline((0.1138, 0.5954), (0.1562, 0.8146), (0.0881, 0.7752))
    Spline(l5 @ 1, l6 @ 0, tangents=(l5 % 1, l6 % 0), tangent_scalars=(2, 2))
    l7 = Line((0.0692, 0.7808), (0.0000, 0.9167))
    TangentArc(l6 @ 1, l7 @ 0, tangent=l6 % 1)
    mirror(l.edges(), about=Plane.YZ)


## BuildPart, BuildSketch, BuildLine

In [ ]:
show(b)

In [ ]:
show(s)

In [ ]:
show(l)

## BuildPart.part, BuildSketch.sketch, BuildLine.line

In [ ]:
show(b.part)

In [ ]:
show(s.sketch)

In [ ]:
show(l.line)

## BuildPart.part.wrapped, BuildSketch.sketch.wrapped, BuildLine.line.wrapped

In [ ]:
show(b.part.wrapped)

In [ ]:
show(s.sketch.wrapped)

In [ ]:
show(l.line.wrapped)

## ShapeLists

In [ ]:
show(b.solids()[0:2])

In [ ]:
show(b, b.faces()[0:2], transparent=True)

In [ ]:
show(b, b.wires()[0:2], transparent=True)

In [ ]:
show(b, b.edges()[0:2], transparent=True)

In [ ]:
show(b, b.vertices()[0:2], transparent=True)

## Direct API

In [ ]:
show(b.solids().sort_by(Axis.X)[-1])

In [ ]:
show(b, b.faces().sort_by(Axis.Z)[-1], transparent=True)

In [ ]:
show(b, b.wires().sort_by(Axis.Z)[-1], transparent=True)

In [ ]:
show(b, b.edges().sort_by(Axis.Z)[-1], transparent=True)

In [ ]:
show(b, b.vertices().sort_by(Axis.Z)[-1], transparent=True)

In [ ]:
show(b.solids().sort_by(Axis.X)[-1].wrapped)

In [ ]:
show(b, b.faces().sort_by(Axis.Z)[-1].wrapped, transparent=True)

In [ ]:
show(b, b.wires().sort_by(Axis.Z)[-1].wrapped, transparent=True)

In [ ]:
show(b, b.edges().sort_by(Axis.Z)[-1].wrapped, transparent=True)

In [ ]:
show(b, b.vertices().sort_by(Axis.Z)[-1].wrapped, transparent=True)

## Tutorial

### Tutorial 1

In [ ]:
with BuildPart() as example:
    Cylinder(radius=10, height=3)
    with BuildSketch(example.faces().sort_by()[-1]):
        RegularPolygon(radius=7, side_count=6)
        Circle(radius=4, mode=Mode.SUBTRACT)
    extrude(amount=-2, mode=Mode.SUBTRACT)
    fillet(
        example
         .edges()
         .filter_by(GeomType.CIRCLE)
         .sort_by(SortBy.RADIUS)[-2:]
         .sort_by(Axis.Z), 
         radius=1
    )
show(example)

### Tutorial 2

In [ ]:
pip_count = 6

lego_unit_size = 8
pip_height = 1.8
pip_diameter = 4.8
block_length = lego_unit_size * pip_count
block_width = 16
base_height = 9.6
block_height = base_height + pip_height
support_outer_diameter = 6.5
support_inner_diameter = 4.8
ridge_width = 0.6
ridge_depth = 0.3
wall_thickness = 1.2

In [ ]:
with BuildPart() as lego:
    with BuildSketch():
        perimeter = Rectangle(block_length, block_width)
        offset(
            perimeter,
            amount=-wall_thickness,
            kind=Kind.INTERSECTION,
            mode=Mode.SUBTRACT,
        )
        with GridLocations(0, lego_unit_size, 1, 2):
            Rectangle(block_length, ridge_width)
        with GridLocations(lego_unit_size, 0, pip_count, 1):
            Rectangle(ridge_width, block_width)
        Rectangle(
            block_length - 2 * (wall_thickness + ridge_depth),
            block_width - 2 * (wall_thickness + ridge_depth),
            mode=Mode.SUBTRACT,
        )
        with GridLocations(lego_unit_size, 0, pip_count - 1, 1):
            Circle(support_outer_diameter / 2)
            Circle(support_inner_diameter / 2, mode=Mode.SUBTRACT)
    extrude(amount=base_height - wall_thickness)
    with Locations(
        Plane(origin=(0, 0, lego.vertices().sort_by(Axis.Z)[-1].Z), z_dir=(0, 0, 1)).location
    ):
        Box(
            block_length,
            block_width,
            wall_thickness,
            align=(Align.CENTER, Align.CENTER, Align.MIN),
        )
    with Locations(lego.faces().sort_by(Axis.Z)[-1]):
        with GridLocations(lego_unit_size, lego_unit_size, pip_count, 2):
            Cylinder(
                radius=pip_diameter / 2, height=pip_height, align=(Align.CENTER, Align.CENTER, Align.MIN)
            )


In [ ]:
show(lego)